# Getting started with Amazon Braket Hybrid Jobs

This tutorial shows how to run your first Amazon Braket Hybrid Job. To get started, we consider small circuits with only one qubit and one gate.


## Learning outcomes
* Get setup to run your first hybrid job
* Write an algorithm script to run on Braket Hybrid Jobs
* Understand how to run scripts or functions
* Create a hybrid job on Braket simulators or QPUs
* Monitoring the hybrid job state
* Save results from a hybrid job
* Using a specific AWS session
* Running hybrid jobs with priority on QPUs
* Use local hybrid jobs to quickly test and debug scripts
* Create a Braket Hybrid Job using the Braket console

## Getting setup to run Braket Hybrid Jobs

When you use Amazon Braket Hybrid Jobs for the first time, you need to create an IAM role with the right [permissions](https://docs.aws.amazon.com/braket/latest/developerguide/braket-manage-access.html#about-amazonbraketjobsexecution). This role allows your hybrid job to perform actions on your behalf while it is executing your algorithm, for instance, access S3 to return the results of your hybrid job. To create the role or check if you already have one please visit the Permissions tab from the left menu of the Braket Console.

## Writing an algorithm script

To create a Braket Hybrid Job, you first need a Python script to run. In this example, it's contained in `algorithm_script.py`. The script is printed in the code cell below for convenience. 

As shown, each of the circuits has only one $X$ rotation gate with a random angle. The circuit is repeated five times with different random rotations. Note that the algorithm script does not specify the backend Amazon Braket device ARN explicitly. Instead, it is provided through environment variables such as `os.environ["AMZN_BRAKET_DEVICE_ARN"]` that are passed to the algorithm script when creating the hybrid job. 

#### This block is a copy of the algorithm script.

```python

import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result
from braket.tracking import Tracker

t = Tracker().start()

print("Test hybrid job started!")

# Use the device declared in the creation script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_circuit = Circuit().rx(0, angle)

    task = device.run(random_circuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list, "estimated cost": t.qpu_tasks_cost() + t.simulator_tasks_cost()})

print("Test hybrid job completed!")
```

## Creating your first hybrid job

Once the script is finalized, you can create a Braket Hybrid Job with `AwsQuantumJob`. When the hybrid job is created, Amazon Braket starts the hybrid job instance (based on EC2) and spins up a Docker container to run your algorithm script. Other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks to learn more about how to customize your jobs

This example uses the following inputs for `AwsQuantumJob`:
- <b>device</b>: The ARN of the Braket simulator or QPU to use in the hybrid job. It will be passed as an environment variable to the algorithm script. 
- <b>source_module</b>: The path to a file or a Python module that contains your algorithm script. It will be uploaded to the container for running the Braket Hybrid Job.
- <b>wait_until_complete (optional)</b>: If True, the function call will wait until the Braket Hybrid Job is completed and will additionally print logs to the local console. Otherwise, it will run asynchronously. The default is False.

In [4]:
from braket.aws import AwsDevice, AwsQuantumJob
from braket.devices import Devices

In [3]:
# This cell should take about 5 mins
job = AwsQuantumJob.create(
    device=Devices.Amazon.SV1,
    source_module="algorithm_script.py",
    wait_until_complete=True,
)

Initializing Braket Job: arn:aws:braket:<region>:<account-id>:job/<job-name>
...............................................
2021-11-10 20:55:24,467 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {},
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {},
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {},
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "BraketJob-<account-id>-<job-name>",
    "log_level": 20,
    "master_hostname": "algo-1",
    "model_dir": "/opt/ml/model",
    "module_dir": "/opt/ml/code",
    "module_name": "braket_container",
    "network_interface_name": "eth0",
    "num_cpus": 2,
    "num_gpus": 0,
    "output_data_dir": "/opt/ml/output/data",
    "output_dir": "/opt/ml/output",
    "output_intermediate_dir": "/opt/ml/output/intermediate",
    "resour

In this example, the algorithm is defined by a single file, so the `source_module` is `algorithm_script.py`. Depending on your application, there are other options for setting the source module. For example, if you wish to only execute a part of `algorithm_script.py` at the start of a Braket Hybrid Job, you can package that part to be a `starting_function()`. Then assign the function as the entry point by adding the `entry_point` input argument.

In [4]:
source_module = "algorithm_script.py"
entry_point = "algorithm_script:starting_function"

If your algorithm script requires other dependencies, you can put them all in one folder, say the `algorithm_folder`. The input arguments would then be

In [5]:
source_module = "algorithm_folder"
entry_point = "algorithm_folder.algorithm_script:starting_function"

## Checking hybrid job state and loading results

The status of a Braket Job can be checked by calling `job.state()`. The state will be one of "QUEUED", "RUNNING", "FAILED", "COMPLETED", "CANCELLING", or "CANCELLED". 

In [6]:
job.state()

'COMPLETED'

Once completed, the result can be retrieved using `job.result()`. Logs and metadata are also accessible via `job.logs()` and `job.metadata()`. If you lose the reference to the hybrid job object, you can always reinstantiate it using your hybrid job ARN as `job=AwsQuantumJob("your-job-arn")`. The ARN of a hybrid job can be found in the Amazon Braket Console. By default the ARN of a hybrid job will be "`arn:aws:braket:<region>:<account_id>:job/<job_name>`". 

In [7]:
results = job.result()  # will return once job.state() = "COMPLETED", should be 6 minutes
print("counts: ", results["counts"])
print("angles: ", results["angles"])

counts:  [{'1': 12, '0': 88}, {'0': 42, '1': 58}, {'0': 46, '1': 54}, {'0': 100}, {'0': 100}]
angles:  [-0.6634809825751307, 1.8729107298836103, -1.8816578492668359, -0.04308463076559567, 0.03224990551866221]


In [8]:
# print(job.logs()) # uncomment to print logs

You can also download the result to a local directory.

In [9]:
job.download_result()  # download hybrid job result to local directory

In [10]:
print("Quantum Task Summary")
print(job.result()['task summary'])
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run quantum tasks in this hybrid job: {job.result()['estimated cost']} USD")

Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 500, 'tasks': {'COMPLETED': 5}, 'execution_duration': 0.081, 'billed_execution_duration': 15.0}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run tasks in this job: 0.01875 USD


## Increase performance by running hybrid jobs on QPUs

With Braket Hybrid Jobs, you can run hybrid algorithms on all QPUs available through Amazon Braket. When you select a QPU as your device, your hybrid job will have priority access for the duration of your hybrid job. Quantum tasks created as part of your hybrid job will be executed ahead of other tasks in the device queue. This reduces the risk of certain tasks being delayed or drifting calibrations on the device. To secure priority on a device your job needs to wait for the device to complete running or already queued jobs. Before submitting the job on a device, you can check the queue depth for the hybrid jobs. To check the number of hybrid jobs queued on the device call `device.queue_depth().jobs`.

In [7]:
# Select the device on which you will be submitting your hybrid job.
device = AwsDevice(Devices.Rigetti.AspenM3)
device.queue_depth().jobs

'2'

You can seamlessly swap the SV1 simulator for a QPU by changing the device argument in `AwsQuantumJob.create()`. For instance, the code below will create a hybrid job with priority access on the Rigetti Aspen-M-3 device:

<div class="alert alert-block alert-info">
    <b>Note:</b> The following cell uses the Rigetti Aspen-M-3 device; before you uncomment and run it, make sure the device is currently available. You can find QPU availability windows on the <a href="https://us-west-1.console.aws.amazon.com/braket/home?region=us-west-1#/devices">Devices page</a> in the Amazon Braket Console
</div>

In [8]:
# qpu_job = AwsQuantumJob.create(
#     device=Devices.Rigetti.AspenM3,
#     source_module="algorithm_script.py",
#     wait_until_complete=False,
# )

You can check the position of your hybrid job in the queue by calling job.queue_position().queue_position. The queue position is only returned when the job is in "QUEUED" state, else None is returned. You can also check why the queue position value is not returned by calling calling `job.queue_position().message`. Here, `job` is the variable to which you assign your job creation. 

In [9]:
# qpu_job.queue_position().queue_position

'3'

When you create the hybrid job, Amazon Braket will wait for the QPU to become available before initializing the hybrid job. Note that the Braket Hybrid Job will automatically select the AWS region where the device is available. As mentioned earlier, the specified device is provided to the hybrid job in the environment variable `AMZN_BRAKET_DEVICE_ARN`; the script `algorithm_script.py` uses this variable to choose the Braket device to use.

In variational algorithms, there are usually optimization processes that update parameters of a fixed parametrized circuit. When executing such algorithms on a supported QPU with Hybrid Jobs, parametric compilation can improve the performance of the hybrid jobs. All you need to do is to submit the parametrized circuit using free parameters in the algorithm script. Braket will compile the circuit once and manage the compiled circuit of the Hybrid Job. There is no recompilation for subsequent parameter updates to the same circuit, resulting in faster runtimes. An example of algorithm script that uses a parametrized circuit is in `algorithm_script_parametrized_circuit.py`. Use this algorithm script in the place of `algorithm_script.py` and submit a hybrid job to a supported Braket QPU to use parametric compilation. See the Amazon Braket developer guide to learn more about [the usage of free parameters](https://docs.aws.amazon.com/braket/latest/developerguide/braket-constructing-circuit.html#braket-gates) and [parametric compilation with Hybrid Jobs](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html).

## AWS Sessions

You can customize the default location where Braket Hybrid Jobs saves and loads results in Amazon S3 by providing the AWS session information. The name of the S3 bucket needs to start with "amazon-braket-", and it must be in the same region as the hybrid job being created.

In [11]:
from braket.aws import AwsSession

# Set Amazon S3 bucket
aws_session = AwsSession(default_bucket="amazon-braket-bucket-name")

To create a Braket Hybrid Job with this S3 bucket, pass `aws_session` as an argument to ` AwsQuantumJob.create()`:

```python
job = AwsQuantumJob.create(
    device=Devices.Amazon.SV1,
    source_module="algorithm_script.py",
    aws_session=aws_session # using specific S3 bucket
)
```

## Debugging with local Braket Hybrid Jobs

For faster testing and debugging of your code, you can run a hybrid job locally in your own environment. This feature requires Docker to be installed in your local environment. Amazon Braket notebooks have Docker pre-installed, so you can test local hybrid jobs in hosted notebooks instantly. To install Docker in your local environment, follow these [instructions](https://docs.docker.com/get-docker/). When a local hybrid job is created for the first time, it will take longer because it needs to build the container. The subsequent runs will be faster. Note that local hybrid jobs will not be visible in the Amazon Braket Console.

To run a hybrid job in local mode, make sure the Docker daemon is running, and then simply create a `LocalQuantumJob` instead of an `AwsQuantumJob`. Local hybrid jobs always run synchronously and display the logs.

In [12]:
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min
job = LocalQuantumJob.create(
    device=Devices.Amazon.SV1,
    source_module="algorithm_script.py",
)

Using the short-lived AWS credentials found in session. They might expire while running.
Boto3 Version:  1.18.33
Beginning Setup
Completed 50.5 KiB/50.5 KiB (72.5 KiB/s) with 1 file(s) remaining
...............................................
...............................................
Running Code As Subprocess
Test job started!!!!!
Counter({'0': 79, '1': 21})
Counter({'0': 84, '1': 16})
Counter({'0': 91, '1': 9})
Counter({'0': 85, '1': 15})
Counter({'1': 94, '0': 6})
Test job completed!!!!!
Code Run Finished


In [13]:
print("Quantum Task Summary")
print(job.result()['task summary'])
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run quantum tasks in this hybrid job: {job.result()['estimated cost']} USD")

Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 500, 'tasks': {'COMPLETED': 5}, 'execution_duration': 0.046, 'billed_execution_duration': 15.0}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run tasks in this job: 0.01875 USD


## Creating a Braket Hybrid Job from the Braket console

Besides creating a Braket Hybrid Job programmatically using `AwsQuantumJob.create`, there is also an option to create a hybrid job in the Braket console. Follow [this link](https://us-west-2.console.aws.amazon.com/braket/home#/job/create) to the \"Create hybrid job\" page. First, we need to give our new hybrid job the algorithm script. The script can be uploaded directly from your computer in the console or selected from files that are uploaded to S3. Here you also have the option to provide input data and parameters to your algorithm. After the algorithm is specified, you have the option to skip directly to the review and create page.

<div align="center"><img src="console_figures/1-algorithm.png"/></div>

Next, you can choose a name for your hybrid job or use the default name. You also have the option to change the default execution role and the default S3 location for uploads.

<div align="center"><img src="console_figures/2-basic.png"/></div>

Third, select a container environment for your job. The default "PennyLane" container is enough for the example in this notebook. For information about using other pre-built or custom containers, see the [Pennylane](../2_Using_PennyLane_with_Braket_Hybrid_Jobs/Using_PennyLane_with_Braket_Hybrid_Jobs.ipynb) and the [BYOC](../3_Bring_your_own_container/bring_your_own_container.ipynb) example notebooks. 

<div align="center"><img src="console_figures/3-environment.png"/></div>

You may also select a Braket on-demand simulator or QPU for your job and configure the execution settings. You also have the option to customize the default locations for checkpoints and output data. We dive deeper into these advanced use cases in other [example notebooks](../2_Using_PennyLane_with_Braket_Hybrid_Jobs/Using_PennyLane_with_Braket_Hybrid_Jobs.ipynb).

<div align="center"><img src="console_figures/4-execution.png"/></div>

After finishing all the settings, you can review your selections before submitting your hybrid job. You can now create the hybrid job by clicking the \"Create hybrid job\" button or return to any earlier step to make edits. We can now view the progress in the Braket console.

<div align="center"><img src="console_figures/5-review.png"/></div>

## Summary

In this tutorial, we have created our first Braket Job with a simple batch of five circuits using the Amazon Braket SDK and, as an alternative, from the Braket console. We learned how to change the Amazon S3 folder and the AWS region for a job, and how to save results, and how to retrieve queue depth for a device, and queue position for the hybrid job. We learned how to seamlessly change the device to run on simulators or QPUs. We used local mode to quickly test code. Finally, we created the same job using the Braket console.